In [1]:
from netCDF4 import Dataset
import matplotlib as pyplot
import numpy as np
import xarray as xr
from osgeo import gdal, osr, gdal_array
import rasterio

C:\Users\rmcd\anaconda3\envs\gridmet_3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\rmcd\anaconda3\envs\gridmet_3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
#=========================================================
#            MACAV2METDATA FILE PARAMETERS
#=========================================================
dirPath='http://thredds.northwestknowledge.net:8080'
fileName='/thredds/dodsC/MET/tmmx/tmmx_2019.nc'

#--------------------------------------------------------
#   FORM FILENAME AND GET HANDLE TO FILE AND DATA
#--------------------------------------------------------
fullfilename= dirPath+fileName
print(fullfilename)

http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/tmmx/tmmx_2019.nc


In [3]:

ds = xr.open_dataset(fullfilename)
# df = ds.to_dataframe()
# print(ds.attrs)
lathandle=ds['lat']
lonhandle=ds['lon']
timehandle=ds['day']
datahandle=ds['air_temperature']
crshandle=ds['crs']
print(crshandle)
print(ds.attrs)
print(ds.attrs['geospatial_bounds'])
print(ds.attrs['geospatial_lon_min'])
print(ds.attrs['geospatial_lat_max'])
print(ds.attrs['geospatial_lat_resolution'])
print(ds.attrs['geospatial_lon_resolution'])
lonmin = float(ds.attrs['geospatial_lon_min'])
latmax = float(ds.attrs['geospatial_lat_max'])
lonres = float(ds.attrs['geospatial_lon_resolution'])
latres = float(ds.attrs['geospatial_lon_resolution'])
print(type(lonmin))
print(datahandle.attrs)
print(datahandle.sizes)
print(datahandle.coords)
dayshape,lonshape,latshape = datahandle.values.shape
print(dayshape,lonshape,latshape)
datahandle.values[85-1,:,:].shape
print(lathandle.values.shape)
print(type(lathandle.values))
print(datahandle.dtype)
print(np.isfortran(datahandle.values))

<xarray.DataArray 'crs' (crs: 1)>
array([1])
Coordinates:
  * crs      (crs) int32 1
Attributes:
    long_name:                    WGS 84
    grid_mapping_name:            latitude_longitude
    longitude_of_prime_meridian:  0.0
    semi_major_axis:              6378137.0
    inverse_flattening:           298.257223563
    GeoTransform:                 -124.7666666333333 0.041666666666666 0  49....
    spatial_ref:                  GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["...
OrderedDict([('geospatial_bounds_crs', 'EPSG:4326'), ('Conventions', 'CF-1.6'), ('geospatial_bounds', 'POLYGON((-124.7666666333333 49.400000000000000, -124.7666666333333 25.066666666666666, -67.058333300000015 25.066666666666666, -67.058333300000015 49.400000000000000, -124.7666666333333 49.400000000000000))'), ('geospatial_lat_min', '25.066666666666666'), ('geospatial_lat_max', '49.40000000000000'), ('geospatial_lon_min', '-124.7666666333333'), ('geospatial_lon_max', '-67.058333300000015'), ('geospatial_lon_res

https://rasterio.readthedocs.io/en/stable/api/rasterio.transform.html#rasterio.transform.from_origin
Return an Affine transformation for a georeferenced raster given the coordinates of its upper left corner west, north and pixel sizes xsize, ysize.

https://gis.stackexchange.com/questions/279953/numpy-array-to-gtiff-using-rasterio-without-source-raster

See also for perhaps some tips: https://github.com/Solcast/netcdf-tiff/blob/master/src/goes16_converter.py


In [16]:
%matplotlib inline
from rasterio.transform import from_origin
lat = lathandle.values
lon = lonhandle.values
# transform = from_origin(472137, 5015782, 0.5, 0.5)
transform = from_origin( lonmin, latmax, lonres, latres)
res = (lon[-1] - lon[0])/lon.shape[0]
print(res)
transform2 = from_origin(lon[0]-res/2, lat[-1]+res/2, res, res) 
print(transform, transform2)
new_dataset = rasterio.open(r'C:\Users\rmcd\Documents\oNHM\GeospatialFabric_1\nhru\unzip\test1.tif', 'w', driver='GTiff',
                            height = lonshape, width = latshape,
                            count=1, dtype=str(datahandle.dtype),
                            crs={'init': 'epsg:4326'},
                            transform=transform)
# vals = np.transpose(datahandle.values, [1,2,0])
# vals2 = vals[:,:,85-1]
vals = datahandle.values[85-1, :, :]
print(vals.shape)
# im = np.transpose(vals, [1,2,0])
# fa = np.asfortranarray(vals)
# ca = np.asanyarray(vals, order='C')
ud = np.flipud(vals)
# at = np.transpose(vals)
# atf = np.rot90(at)
new_dataset.write(ud, 1)
# new_dataset.warp.transform
# print(new_dataset.transform)
# pyplot.imshow(new_dataset.read(1), cmap='pink')
# pyplot.show()
new_dataset.close()

0.04163660413660413
| 0.04, 0.00,-124.77|
| 0.00,-0.04, 49.40|
| 0.00, 0.00, 1.00| | 0.04, 0.00,-124.79|
| 0.00,-0.04, 49.42|
| 0.00, 0.00, 1.00|
(585, 1386)


In [11]:
dataset = rasterio.open(r'C:\Users\rmcd\Documents\oNHM\GeospatialFabric_1\nhru\unzip\test1.tif')
print(dataset.bounds)
print(dataset.transform)
dataset.close()

BoundingBox(left=-124.78748493540165, bottom=-8.287515031265023, right=-100.43007151548824, top=49.420818302068305)
| 0.04, 0.00,-124.79|
| 0.00,-0.04, 49.42|
| 0.00, 0.00, 1.00|


In [ ]:
import osr
src_ds = gdal.Open(r'C:\Users\rmcd\Documents\oNHM\GeospatialFabric_1\nhru\unzip\tmmx_2019.nc')
Projection = osr.SpatialReference()
print(src_ds.GetProjectionRef())
Projection.ImportFromWkt(src_ds.GetProjectionRef())
# GeoT = src_ds.osr.SpatialTransform()
# print(GeoT)
print(type(Projection))
if src_ds is None:
    print("Open failed")
subdatasets = ds_in.GetSubDatasets()
variables = []
for subdataset in subdatasets:
    variables.append(subdataset[1].split(" ")[1])
print(src_ds)

In [ ]:
def GetnetCDFInfobyName(in_filename, var_name):
    '''Function to read the original file's projection '''
    #Open netCDF file
    src_ds = gdal.Open(in_filename)
    if src_ds is None:
        print("Open failed")
        #sys.exit()
    #if src_ds.GetSubDatasets() > 1:
    #if exists more than one var in netCDF...
    subdataset = 'NETCDF:"'+ in_filename +'":'+ var_name
    print(subdataset)
    src_ds_sd = gdal.Open(subdataset)
    # read info
    NDV = src_ds_sd.GetRasterBand(1).GetNoDataValue()
    xsize = src_ds_sd.RasterXSize
    ysize = src_ds_sd.RasterYSize
    GeoT = src_ds_sd.osr.SpatialTransform()
    Projection = osr.SpatialReference()
    Projection.ImportFromWkt(src_ds_sd.GetProjectionRef())
    #close subdataset and whole dataset
    src_ds_sd = None
    src_ds = None
    # read data using xarray
    xr_ensemble = xr.open_dataset(in_filename)
    data = xr_ensemble[var_name]
    data = np.ma.masked_array(data, mask=data==NDV, fill_value=NDV)
        
    return NDV, xsize, ysize, GeoT, Projection, data

In [ ]:
import os
unzip_dir = r'C:\Users\rmcd\Documents\oNHM\GeospatialFabric_1\nhru\unzip'
os.chdir(unzip_dir)
tfile = 'tmmx_2019.nc'
ds = xr.open_dataset(tfile)
print(ds)
filename_w_ext = os.path.basename(fullfilename)
print(filename_w_ext)
ds.to_netcdf(filename_w_ext)
# print(gdal.Open(filename_w_ext))

In [ ]:
print(os.getcwd(), filename_w_ext)
ds_in = gdal.Open(filename_w_ext)
print(ds_in)

subdatasets = ds_in.GetSubDatasets()
print(subdatasets)
variables = []
for subdataset in subdatasets:
    variables.append(subdataset[1].split(" ")[1])

print(variables)



# print(GetnetCDFInfobyName(filename_w_ext, 'air_temperature'))